# Data Splitting and Modelling

Author: Gillian A. McGinnis, final-semester M.S. Information Science - Machine Learning  
The University of Arizona College of Information  
INFO 698 - Capstone  
Start date: 21 October 2025  
Last updated: 25 November 2025

In [ ]:
"""
Module providing code for test/train split and sliding window creation. Relies on 01_clean.ipynb completion.
"""

## Setup

In [ ]:
var_of_interest = "obstruction_ro"

### Packages

In [ ]:
# GPU Setup
%load_ext cudf.pandas
import pandas as pd
import cudf
import cupy as cp

import cuml.accel
cuml.accel.install()

from sklearn.model_selection import TimeSeriesSplit, train_test_split, RandomizedSearchCV, TunedThresholdClassifierCV
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, precision_recall_curve, make_scorer, roc_auc_score, auc
# For help with model tuning
from sklearn.base import clone

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

from google.colab import userdata
gh_pat = userdata.get('gh_pat')
gh_user = userdata.get('gh_user')
gh_repo = userdata.get('gh_repo')
import os
if not os.path.exists(f'{gh_repo}'):
  print("Cloning repo...")
  repo_url = f'https://{gh_pat}@github.com/{gh_user}/{gh_repo}'
  !git clone {repo_url}

if os.getcwd() == '/content':
    print("Changing wd...")
    os.chdir(f'{gh_repo}/code')

# # Verify the current working directory
print(f"Current working directory is: {os.getcwd()}")

del gh_pat, gh_user, gh_repo

In [ ]:
# # from google.colab import drive
# # drive.mount('/content/drive')

# from google.colab import userdata
# gh_pat = userdata.get('gh_pat')
# gh_repo = userdata.get('gh_repo')
# repo_url = f'https://{gh_pat}@github.com/{gh_repo}'
# !git clone {repo_url}

In [ ]:
# import os
# if os.getcwd() == '/content':
#     print("Changing wd...")
#     os.chdir('info-698-capstone/code')

# # # Verify the current working directory
# print(f"Current working directory is: {os.getcwd()}")

In [ ]:
# General packages
import numpy as np
import xgboost as xgb
# import pandas as pd
# import matplotlib.pyplot as plt
from scipy.stats import randint, uniform
# from sklearn.model_selection import TimeSeriesSplit, train_test_split, RandomizedSearchCV, TunedThresholdClassifierCV
# from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, precision_recall_curve, make_scorer, roc_auc_score

# from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, make_scorer, roc_auc_score
# from cuml.metrics import precision_recall_curve

# For saving models
import joblib

# For data importing and exporting
from helper_utils import get_path, model_path

In [ ]:
## (Optional chunk)
# Current session information

# From StackOverflow,
# https://stackoverflow.com/a/62128239/23486987
try:
    import session_info
except:
    !pip install session_info
    import session_info
# !pip install session_info
# import session_info
session_info.show(dependencies=False)

In [ ]:
# # To make it easier to tell when processes have completed -- can delete later
# # From StackOverflow,
# # https://stackoverflow.com/a/62128239/23486987
# try:
#     from playsound3 import playsound
# except:
#     !pip install playsound3
#     from playsound3 import playsound

# Google colab compatible:
# https://stackoverflow.com/a/68582785/23486987
from IPython.display import Audio, display

def play_chime():
  return Audio(get_path('completed.mp3', 'code'), autoplay=True)


In [ ]:
# Set seed
np.random.seed(42)
cp.random.seed(42)

In [ ]:
# Make sure GPU active
# !nvidia-smi
import torch
if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("Warning: GPU not found!")

### Data

In [ ]:
# united_water = pd.read_parquet('data/clean/water_nocal.parquet')
# data_cal = pd.read_parquet('data/clean/calibration.parquet')
# data_cal = data_cal.rename(columns={'weir_level':'weir_level_cal'})

# united_soil = pd.read_parquet('data/clean/soil.parquet')

united_water = pd.read_parquet(get_path('clean/water_nocal.parquet'))
united_soil = pd.read_parquet(get_path('clean/soil.parquet'))

# united_water = pd.read_parquet('data/clean/water_nocal.parquet')
data_cal = pd.read_parquet(get_path('clean/calibration.parquet'))
data_cal = data_cal.rename(columns={'weir_level':'weir_level_cal'})

# united_soil = pd.read_parquet('data/clean/soil.parquet')

In [ ]:
data_cal.info()

### Cleanup

Small amount of data wrangling for memory improvements (some as a consequence of importing).

#### Memory improvements

In [ ]:
# Select columns of interest
data_water = united_water.drop(columns=['raw_rain', 'chk_note_rain', 'chk_fail_rain', 'chk_note_ro', 'chk_fail_ro', 'comment_ro', 'source_ro'])

# Cleanup
del united_water

# Remove duplicate entries
data_water = data_water.reset_index().drop_duplicates(keep='first').set_index('datetime')

In [ ]:
water_drops = ['level_ro', 'obstruction_ro', 'gap_fill_ro', 'weir_cleaning_ro', 'spike_ro', 'calibration_ro']
water_drops.remove(var_of_interest)

data_water = data_water.drop(water_drops, axis=1)

del water_drops

data_water.info(memory_usage="deep")

In [ ]:
united_soil['sample'] = united_soil['sample'].astype('category')

## Prepare

In [ ]:
# temp_subset_start= None
# temp_subset_end = None
# ### Note ###
# # REMOVE this later -- just a smaller subset for feature engineering testing!!!
# # temp_subset_start = '2000-01-01 00:00:00'
temp_subset_start = '2001-02-01 00:00:00'
temp_subset_end = '2011-12-31 23:59:59'
# ######
data_water = data_water[temp_subset_start:temp_subset_end]

## Feature Engineering

### Distance from Event

In [ ]:
def timesince_feat(input_df, input_col, input_unit):
    # output_df = input_df.copy()#[input_col].to_frame()
    output_df = input_df
    instances = output_df[input_col].notna()
    # Create groupings based on most recent instance
    group_id = instances.cumsum()
    # Exclude the first grouping
    # otherwise it assumes there was an event just prior to the first entry
    group_id = group_id.replace(0, np.nan)
    # Create new column to count the distance in days since the point
    # which resets to 0 at each new point
    output_df['timestamp'] = pd.to_datetime(output_df.index)
    # Get start timestamp of the group
    output_df['ts_start'] = output_df.groupby(group_id)['timestamp'].transform('min')
    # Calculate the distance
    if input_unit == "minutes":
        # output_df[f"minsince_{input_col}"] = (output_df['timestamp'] - output_df['ts_start']).dt.total_seconds().div(60).astype('Int32')
        output_df[f"minsince_{input_col}"] = (output_df['timestamp'] - output_df['ts_start']).dt.total_seconds().div(60).astype(np.float32)
        # output_df[f"minsince_{input_col}"] = output_df[f"minsince_{input_col}"].astype(np.float32)
    elif input_unit == "days":
        # output_df[f"daysince_{input_col}"] = (output_df['timestamp'] - output_df['ts_start']).dt.days.astype('Int32')
        output_df[f"daysince_{input_col}"] = (output_df['timestamp'] - output_df['ts_start']).dt.days.astype(np.float32)
        # output_df[f"minsince_{input_col}"] = output_df[f"minsince_{input_col}"].astype(np.float32)
        # output_df[f"daysince_{input_col}"] = output_df[f"daysince_{input_col}"].astype('Int32')
    # Remove extra cols
    output_df = output_df.drop(columns=['timestamp', 'ts_start'])
    return output_df

#### Rain
Create feature which tracks how recent a rain event occurred.

In [ ]:
data_water = timesince_feat(data_water, 'ra_rain', "minutes")

### Rain event

Keep track of cumulative rainfall during a specific event.

In [ ]:
# Create index of instances where there is a data point
rain_event = (data_water['ra_rain'].isnull() & ((data_water['minsince_ra_rain'] >= 5.0) & (data_water['minsince_ra_rain'] != 0)))
# Create groupings based on most recent instance
rain_event_id = rain_event.cumsum()
# Create new column to count number of records since the point
# which resets to 0 at each new point
data_water['eventsum_ra_rain'] = data_water.groupby(rain_event_id)['ra_rain'].cumsum()

del rain_event, rain_event_id

### Decay

In [ ]:
def decay_feat(input_df, input_col, input_dec_rate = -0.1):
    output_df = input_df
    if f"minsince_{input_col}" not in output_df.columns:
        output_df = timesince_feat(input_df = output_df, input_col = input_col, input_unit = "minutes")
    # Update for GPU for overflow fix
    output_df[f"minsince_{input_col}"] = output_df[f"minsince_{input_col}"].astype(np.float64)

    output_df[f"decayrate{input_dec_rate}_{input_col}"] = np.exp(input_dec_rate * output_df[f"minsince_{input_col}"]).astype(np.float32)
    output_df[f"ffill_{input_col}"] = output_df[input_col].ffill()
    output_df[f"decay{input_dec_rate}_{input_col}"] = (output_df[f"ffill_{input_col}"] * output_df[f"decayrate{input_dec_rate}_{input_col}"])

    return output_df

In [ ]:
# Replace NAs in rain with 0
data_water['ra_rain'] = data_water['ra_rain'].fillna(0)

# Apply decay function
data_water = decay_feat(data_water, 'eventsum_ra_rain')

# Drop extra column
# minutes since rain event will be the same as minutes since most recent rain
data_water = data_water.drop('minsince_eventsum_ra_rain', axis=1)

### Lag features

#### Consistent cols

Modify the rows to prevent inappropriate data shifts

In [ ]:
original_indices = data_water.index.copy()

new_index = pd.date_range(start = data_water.index.min(),
                          end = data_water.index.max(),
                          freq = '5min')

# Reindex
data_water = data_water.reindex(new_index)

# Cleanup
del new_index

# # Return
# data_water = data_water.loc[original_indices]
# del original_indices

Get values from other recent time stamps.

In [ ]:
def lag_feats(input_df, input_cols, input_lags):
    output_df = input_df#.copy()
    for col in input_cols:
        for lag in input_lags:
            output_df[f"{col}_lag{lag}"] = output_df[col].shift(lag)
    return output_df

In [ ]:
# Columns to get temporal stats on
cols_to_shift = ['raw_ro', 'ra_rain']

# data at 5-min increments -- lag to record values at 5m, 10m, 15m, 20m, 25m, 30m, 1h, 2h, 3h prior
lags_of_interest = [1, 2, 3, 4, 5, 6, 12, 24, 36]

data_water = lag_feats(data_water, cols_to_shift, lags_of_interest)

# data_water.sample(10)

### Rolling stats

Get stat values from range of recent time stamps.

In [ ]:
def rolling_feats(input_df, input_cols, input_windows, input_mtype = "mean"):
    output_df = input_df#.copy()

    # Create a dummy series of index values (0, 1, 2, ... N) once
    # 'x' represents the position within the dataframe for the regression calculation
    x_series = pd.Series(np.arange(len(output_df)), index=output_df.index)

    for col in input_cols:
        for window in input_windows:
            # 1. Calculate mean and std
            if input_mtype == "mean":
                output_df[f"{col}_rollmean_{window}"] = output_df[col].rolling(window).mean().astype(np.float32)
            elif input_mtype == "sum":
                output_df[f"{col}_rollsum_{window}"] = output_df[col].rolling(window).sum().astype(np.float32)
            elif input_mtype == "both":
                output_df[f"{col}_rollmean_{window}"] = output_df[col].rolling(window).mean().astype(np.float32)
                output_df[f"{col}_rollsum_{window}"] = output_df[col].rolling(window).sum().astype(np.float32)
            output_df[f"{col}_rollstd_{window}"] = output_df[col].rolling(window).std().astype(np.float32)

            # 2. Calculate Slope using vectorized operations
            # Calculate Covariance of Y (data) vs X (index position)
            rolling_cov = output_df[col].rolling(window).cov(x_series)
            # Calculate Variance of X (index position)
            rolling_var_x = x_series.rolling(window).var()
            # Slope = Cov(Y, X) / Var(X)
            output_df[f"{col}_rollslope_{window}"] = (rolling_cov / rolling_var_x).astype(np.float32)

            # Note on edge cases:
            # The initial 'window-1' values for rolling_var_x will be NaN/incorrect.
            # Pandas automatically handles alignment, so the division result will also be NaN where appropriate.
            # This method works very well for standard time series analysis.
    return output_df

In [ ]:
# Inclusive of current point--
# 10m, 15m, 20m, 25m, 30m, 1h, 3h, 6h, 12h, 24h
windows_of_interest = [2, 3, 4, 5, 6, 12, 36, 72, 144, 288]

# data_water = rolling_feats(data_water, ['raw_ro'], windows_of_interest, "mean")
data_water = rolling_feats(data_water, ['raw_ro'], windows_of_interest, "both")
data_water = rolling_feats(data_water, ['ra_rain'], windows_of_interest, "sum")

Change since last value

In [ ]:
data_water['raw_ro_change'] = data_water['raw_ro'].diff()
data_water['ra_rain_change'] = data_water['ra_rain'].diff()
# data_water['raw_ro_rollmean_2']
data_water['raw_ro_rollmean_2_change'] = data_water['raw_ro_rollmean_2'].diff()

In [ ]:
# Revert index
# (adjusted for GPU)

data_water_reset = data_water.reset_index()
index_col_name = data_water_reset.columns[0]
indices_df = original_indices.to_frame(name=index_col_name)

filtered_data_water = cudf.merge(
    data_water_reset,
    indices_df,
    on=index_col_name,
    how='inner'
)
data_water = filtered_data_water.set_index(index_col_name)

del original_indices, filtered_data_water, index_col_name, data_water_reset, indices_df

In [ ]:
# # Return
# data_water = data_water.loc[original_indices]

# del original_indices

## Soil

Pivot the soil data such that each sample has its own columns, and separated by depth.

In [ ]:
# Drop irrelevant column
data_soil_shallow = united_soil.copy().drop('h2o_by_wet_deep', axis=1)
data_soil_shallow['sample'] = data_soil_shallow['sample'].astype('float32')
# Pivot wider
data_soil_shallow = data_soil_shallow.pivot(columns='sample', values='h2o_by_wet_shallow')

# Drop irrelevant column
data_soil_deep = united_soil.copy().drop('h2o_by_wet_shallow', axis=1)

data_soil_deep['sample'] = data_soil_deep['sample'].astype('float32')
# Pivot wider
data_soil_deep = data_soil_deep.pivot(columns='sample', values='h2o_by_wet_deep')

In [ ]:
data_soil = pd.merge(
    data_soil_shallow,
    data_soil_deep,
    left_index = True,
    right_index = True,
    suffixes = ("_shallow", "_deep"),
    how = "outer"
)

del data_soil_shallow, data_soil_deep
del united_soil

## Unite

In [ ]:
data_united = pd.merge(
    data_water,
    # REMOVE LATER
    data_cal[temp_subset_start:temp_subset_end],
    #
    left_index = True,
    right_index = True,
    how = 'outer'
)

data_united = pd.merge(
    data_united,
    # REMOVE LATER
    data_soil[temp_subset_start:temp_subset_end],
    #
    left_index = True,
    right_index = True,
    how = 'outer'
)

### United features

In [ ]:
# Difference compared to calibration point (infrequent)
data_united['diff_ro_cal'] = (data_united['weir_level_cal'] - data_united['raw_ro'])

# Convert to float
data_united['diff_ro_cal'] = data_united['diff_ro_cal'].astype(np.float32)

# Time since last calibration point
data_united = timesince_feat(data_united, 'weir_level_cal', "minutes")

### Temporal features
Modify temporal features to be based on sine and cosine transformations, which allows for the model to be based on the cyclical patterns of time rather than abrupt distances

(e.g., the raw values Day 365 of the year is 'far' from Day 001, but in reality they are very near)

In [ ]:
def temporal_feat(input_df, input_unit):
    output_df = input_df
    if input_unit=='day':
        cycle_length = 365.25
        value = output_df.index.dayofyear
    elif input_unit=='month':
        cycle_length = 12
        value = output_df.index.month
    elif input_unit=='hour':
        cycle_length = 24
        value = output_df.index.hour
    elif input_unit=='minute':
        cycle_length = 60
        value = output_df.index.minute

    output_df[f'{input_unit}_sin'] = np.sin(2 * np.pi * value / cycle_length).astype(np.float32)
    output_df[f'{input_unit}_cos'] = np.cos(2 * np.pi * value / cycle_length).astype(np.float32)

    return output_df

In [ ]:
data_united = temporal_feat(data_united, 'minute')
data_united = temporal_feat(data_united, 'hour')
data_united = temporal_feat(data_united, 'day')
data_united = temporal_feat(data_united, 'month')

In [ ]:
# create features to track soil value staleness
cols_soil = [col for col in data_united.columns if (col.endswith('shallow') | col.endswith('deep'))]

for col in cols_soil:
# for col in data_united.columns:
    # if (col.endswith('shallow') | col.endswith('deep')):
    # data_united = minsince_feat(data_united, col)
    data_united = timesince_feat(data_united, col, "days")

# Extend soil vals
data_united[cols_soil] = data_united[cols_soil].ffill()

del col, cols_soil

## Train/Test split

80/20 initial split, with expanding sliding window for training/validation for hyperparameters, model stability, and feature selection.

In [ ]:
# REMOVE NAs
data_united = data_united.dropna(subset=[var_of_interest])

X_all = data_united.drop(var_of_interest, axis=1).copy()
y_all = data_united[var_of_interest].copy()

In [ ]:
# Fix for inferred later
# y_all = y_all.astype(bool)
y_all = y_all.astype(np.float32)
y_all = y_all.as_gpu_object()

for col in X_all.columns:
  if str(X_all[col].dtype) == ('Int32'):
    X_all[col] = X_all[col].astype(np.float32)

# print(y_all.__class__)
# y_all = cudf.Series.from_pandas(y_all)
print(y_all.__class__)
X_all.info()

In [ ]:
y_len = len(y_all)

print(
    y_len, "\n",
    (round(.2*y_len) + round(.8*y_len)),
    "\nTrain:\t80p of ", y_len, " is ", round(.8*y_len),
    "\nTest:\t20p of ", y_len, " is ", round(.2*y_len),
    sep=""
)

del y_len

Unlike the typical approach for train/test splits, temporal data in this context must _not_ be randomly split as it would lead to severe leakage.

In [ ]:
# Conduct the split
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = 0.2, shuffle=False)
# Conduct an inner split for tuning
# X_train_inner, X_test_inner, y_train_inner, y_test_inner = train_test_split(X_train, y_train, test_size = 0.2, shuffle=False)

# Cleanup
del X_all, y_all

print(
    "Train:\t", len(X_train), "\t", X_train.index[0], "thru", X_train.index[-1],
    "\nTest:\t", len(X_test), "\t", X_test.index[0], "thru", X_test.index[-1]
    # len(x_train), len(x_test), "\n",
    # x_train.index[-1]
)

### Expanding Window

For tuning, an expanding window approach will be used. This is similar to how a model would act once deployed, as it will only gain more data over time.

In [ ]:
# Initialize the split function
tscv = TimeSeriesSplit(n_splits=5)
# print(tscv)

for i, (train_index, val_index) in enumerate(tscv.split(X_train)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={val_index}")
    # print("  Train: index=", mini_x.index[train_index])
    # print(f"  Test:  index={val_index}")
    print("------------------------------------------------------------")

del i, train_index, val_index

## Hyperparameter tuning

As per XGBoosting documentation/tutorials, early stopping with random search for hyperparameter tuning must be iterated upon manually, as `RandomizedSearchCV` does not support using a separate validation set within each CV fold.

Source: https://xgboosting.com/xgboost-early-stopping-with-random-search/


The area under the precision-recall curve (AUC-PR) can be used to evaluate the performance, since it considers a range of classification thresholds.
This is better than an ROC AUC metric since there is greater class imbalance (i.e., `True` is more rare).

Source: https://xgboosting.com/evaluate-xgboost-performance-with-precision-recall-curve/


In [ ]:
# Code modified from
# https://xgboosting.com/xgboost-early-stopping-with-random-search/

# Define hyperparameter distributions for random search
n_est = 1000

param_distributions = {
    'learning_rate': ('uniform', 0.01, 0.3),
    'max_depth': ('choice', [2, 3, 4, 5, 7]),
    'subsample': ('uniform', 0.5, 1.0),
    'colsample_bytree': ('uniform', 0.4, 1.0),
    'scale_pos_weight':('choice', [1, 5, 10, 15, 20]),
    'gamma': ('uniform', 0, 0.5),
    'reg_alpha': ('uniform', 0, 1.0)
}

# # Function to sample parameters based on their distribution type
# def sample_param(distribution):
#     if distribution[0] == 'uniform':
#         return uniform(distribution[1], distribution[2] - distribution[1]).rvs()
#     elif distribution[0] == 'choice':
#         return np.random.choice(distribution[1])
#     else:
#         raise ValueError(f"Unsupported distribution type: {distribution[0]}")


# Define seed again
rng = np.random.default_rng(42)

# Function to sample parameters based on their distribution type
def sample_param(distribution):
    if distribution[0] == 'uniform':
        # Use seeded generator to create the scipy distribution
        return uniform(loc=distribution[1], scale=distribution[2] - distribution[1]).rvs(random_state=rng)
    elif distribution[0] == 'choice':
        # Use seeded generator choice method
        return rng.choice(distribution[1])
    else:
        raise ValueError(f"Unsupported distribution type: {distribution[0]}")

# Configure CV and early stopping
n_splits = 5
# early_stopping_rounds = 10
early_stopping_rounds = 25

# Perform random search with early stopping
# n_iterations = 3
n_iterations = 50
best_params = None
best_score = 0
best_avg_rounds = 0

for _ in range(n_iterations):
    test_scores = []
    best_rounds = []
    optimal_rounds_list = []
    params = {k: sample_param(v) for k, v in param_distributions.items()}

    for train_index, test_index in tscv.split(X_train):
        X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]

        # Split train set into train and validation
        X_train_fold, X_val, y_train_fold, y_val = train_test_split(X_train_fold, y_train_fold, test_size=0.2, shuffle=False)

        # # Prep hyperparam
        # neg_count_fold = (y_train_fold == 0).sum()
        # pos_count_fold = (y_train_fold == 1).sum()

        # Prepare the model
        model = xgb.XGBClassifier(
            n_estimators=n_est,
            learning_rate=params['learning_rate'],
            max_depth=int(params['max_depth']),  # max_depth should be an int
            subsample=params['subsample'],
            colsample_bytree=params['colsample_bytree'],
            objective='binary:logistic',
            random_state=42,
            ## SETTINGS FOR GPU
            seed_per_iteration = True,
            tree_method='hist',
            device='cuda',
            # scale_pos_weight= neg_count_fold/pos_count_fold,
            scale_pos_weight=params['scale_pos_weight'],
            gamma=params['gamma'],
            eval_metric='aucpr',
            reg_alpha=params['reg_alpha'],
            # disable_default_eval_metric=True,
            ##
            n_jobs=-1,
            early_stopping_rounds=early_stopping_rounds # fixed early stopping
        )

        # Fit model on train fold and use validation for early stopping
        model.fit(X_train_fold, y_train_fold, eval_set=[(X_val, y_val)], verbose=False)

        # Find optimal number of iterations
        optimal_rounds_list.append(model.best_iteration)

        # # Predict on test set
        # y_pred_test = model.predict(X_test_fold)
        # # test_score = accuracy_score(y_test_fold, y_pred_test)
        # # test_score = f1_score(y_test_fold, y_pred_test)
        # test_score = f1_score(y_test_fold.to_cupy().get(), y_pred_test)
        # test_scores.append(test_score)

        # ## UDPATE -- using ROC AUC
        # y_pred_test = model.predict_proba(X_test_fold)[:,1]
        # # y_pred_test = y_pred_test.to_cupy()
        # # y_pred_test = cp.array(y_pred_test)
        # y_pred_test = cudf.Series(y_pred_test)
        # # test_score = roc_auc_score(y_test_fold, y_pred_test)
        # test_score = cuml.metrics.roc_auc_score(y_test_fold, y_pred_test)
        # test_scores.append(test_score)
        # ##

        ## Using AUC PR
        # Predict on test set
        y_pred_test = model.predict(X_test_fold)
        # test_score = accuracy_score(y_test_fold, y_pred_test)
        # test_score = f1_score(y_test_fold, y_pred_test)
        prec, rec, _ = precision_recall_curve(y_test_fold.to_cupy().get(), y_pred_test)
        test_score = auc(rec, prec)
        test_scores.append(test_score)

    # Compute average score across all folds
    average_score = np.mean(test_scores)
    average_optimal_rounds = np.mean(optimal_rounds_list)

    if average_score > best_score:
        best_score = average_score
        best_params = params
        best_avg_rounds = int(round(average_optimal_rounds)) # Store the integer average
        ## Maybe??
        # best_f1 = test_score
        # best_model = model


print(f"Best Parameters: {best_params}")
# print(f"Best CV Average Accuracy: {best_score}")
# print(f"CV Average ROC AUC: {best_score:.4f}")
print(f"CV Average: {average_score:.4f}")
print(f"Best Avg Rounds: {best_avg_rounds}")

In [ ]:
# _ = playsound(get_path('completed.mp3', 'code'), block=False)
play_chime()

In [ ]:
# print(f"Best ROC AUC: {test_score:.4f}")
del X_train_fold, X_val, y_train_fold, y_val
# del y_pred_test, test_score
del y_pred_test
del n_splits, early_stopping_rounds, n_iterations, test_scores, best_rounds, optimal_rounds_list
# del neg_count_fold, pos_count_fold
del average_score, average_optimal_rounds

## Save model

In [ ]:
model_name = "rs_xgb_cpu_rocauc_tt_full_aucrc_regz_iter"

if os.path.exists(model_path(model_name)) == False:
    print("Fitting final model...")

    # XGBoost requires int for certain params
    # best_params['max_depth'] = int(best_params['max_depth'])

    # # Prep hyperparam
    # neg_count_it = (y_train_inner == 0).sum()
    # pos_count_it = (y_train_inner == 1).sum()

    final_model = xgb.XGBClassifier(
        n_estimators=best_avg_rounds,
        # n_estimators=100, # avg optimal n_estimators if known, or reasonable default
        learning_rate=best_params['learning_rate'],
        max_depth=int(best_params['max_depth']),
        subsample=best_params['subsample'],
        colsample_bytree=best_params['colsample_bytree'],
        objective='binary:logistic',
        random_state=42,
        ## SETTINGS FOR GPU
        seed_per_iteration = True,
        tree_method='hist',
        device='cuda',
        scale_pos_weight=best_params['scale_pos_weight'],
        gamma=best_params['gamma'],
        reg_alpha=best_params['reg_alpha'],
        ##
        n_jobs=-1
    )

    # final_model.fit(X_train_inner, y_train_inner)
    joblib.dump(final_model, model_path(model_name))

    # del neg_count_it, pos_count_it

    # Local download
    from google.colab import files
    files.download(model_path(model_name))

else:
    print("Importing model from saved files...")
    final_model = joblib.load(model_path(model_name))

## OOF Predictions

In [ ]:
# oof_pred = []
from sklearn.base import clone

oof_pred = np.full(len(y_train), np.nan)

for train_index, test_index in tscv.split(X_train):
  X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
  y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]

  # Prepare the model
  # model = final_model.copy()
  model = clone(final_model)

  model.fit(X_train_fold, y_train_fold)
  # model.pred_proba(X_test_fold, y_test_fold)
  y_pred_test = model.predict_proba(X_test_fold)[:,1]
  # oof_pred.update(y_pred_test, index=test_index)
  oof_pred[test_index] = y_pred_test

valid_mask = ~np.isnan(oof_pred)
y_train_oof = y_train.values[valid_mask]
oof_pred = oof_pred[valid_mask]

del model, X_train_fold, y_train_fold, X_test_fold, y_test_fold, y_pred_test
del valid_mask
# final_model.fit(X_train_inner, y_train_inner)

## Smoothing & Thresholding

Windowing can help smooth predictions by preventing standalone points that differ from their neighbors (e.g., having a sequence of `True` interrupted by one `False`, or vice-versa, both of which are unlikely in this context due to how weir blockages occur).

By default, a threshold of 0.5 will be selected for categorizing a point as `True` or `False`. However, in this context a model more sensitive to `True` may make final results more accurate.

These measures can be found by finding the window and corresponding threshold that maximizes the F1 score.

In [ ]:
def smooth_window(probs, window_size=3):
    return pd.Series(probs).rolling(window=window_size, min_periods=1, center=True).mean().values

In [ ]:
def f1_cust(input_true, input_pred):
  output_true = input_true.copy()
  output_pred = input_pred.copy()
  if not isinstance(output_true, np.ndarray):
    output_true = output_true.get()
  if not isinstance(output_pred, np.ndarray):
    output_pred = output_pred.get()

  output_f1 = f1_score(output_true, output_pred)
  # prec, rec, _ = precision_recall_curve(output_true, output_pred)
  # output_f1 = auc(rec, prec)
  return output_f1

In [ ]:
# prec, rec, _ = precision_recall_curve(y_train_oof.get(), oof_pred)
# # auc(rec, prec)
# print(
#     best_score,
#     auc(rec, prec)
#   )

In [ ]:
from scipy.signal import medfilt


In [ ]:
# Start w no smoothing
best_window_size = 1
# Start w default threshold
threshold = 0.5

y_bin = (oof_pred >= threshold).astype(np.int32)
# y_bin = cp.array(y_bin)
best_f1 = f1_cust(y_train_oof, y_bin)
# print(f"No smoothing, 0.5 threshold:\t{best_f1:.4f}")

best_threshold = threshold
thresholds = np.linspace(0.01, 0.99, 100)
# thresholds = np.linspace(0.01, 0.99, 10)

# Smallest window size
window_min = 1
# Largest window size
window_max = 35
# window_max = 10

print(f"Window 1\tCurr best: W:1\tT:{best_threshold}\t\tF1:{best_f1:.4}")

# Test range of odd window sizes
for current_window in range(window_min, window_max+1, 2):
    # print("Testing window =", current_window)
    # smoothed_preds = smooth_window(oof_pred, window_size=current_window)
    smoothed_preds = medfilt(oof_pred, kernel_size=current_window)

    for threshold in thresholds:
        y_bin = (smoothed_preds >= threshold).astype(np.int32)
        current_f1 = f1_cust(y_train_oof, y_bin)
        # print(f"{threshold:.4}\t{current_f1:.5}")

        if current_f1 > best_f1:
          best_f1 = current_f1
          # print(f"New best F1:{best_f1:.4}")
          best_window = current_window
          best_threshold = threshold
    # print(f"Current best F1:{best_f1:.4}")
    print(f"Window {current_window}\tCurr best: W:{best_window}\tT:{best_threshold:.4}\tF1:{best_f1:.4}")

# oof_pred_adj = smooth_window(oof_pred, window_size=best_window)
oof_pred_adj = medfilt(oof_pred, kernel_size=best_window)
oof_pred_adj = (oof_pred_adj >= best_threshold).astype(int)

f1_rez = f1_cust(y_train_oof, oof_pred_adj)
print(f"Window {best_window}, threshold {best_threshold:.4f}:\t{f1_rez:.4f}")

del current_f1, current_window, threshold, f1_rez, window_min, window_max, y_bin, best_f1

In [ ]:
# Start w no smoothing
best_window_size = 1
# Start w default threshold
threshold = 0.5

y_bin = (oof_pred >= threshold).astype(np.int32)
# y_bin = cp.array(y_bin)
best_f1 = f1_cust(y_train_oof, y_bin)
# print(f"No smoothing, 0.5 threshold:\t{best_f1:.4f}")

best_threshold = threshold
thresholds = np.linspace(0.01, 0.99, 100)
# thresholds = np.linspace(0.01, 0.99, 10)

# Smallest window size
window_min = 1
# Largest window size
window_max = 35
# window_max = 10

print(f"Window 1\tCurr best: W:1\tT:{best_threshold}\t\tF1:{best_f1:.4}")

# Test range of odd window sizes
for current_window in range(window_min, window_max+1, 2):
    # print("Testing window =", current_window)
    smoothed_preds = smooth_window(oof_pred, window_size=current_window)

    for threshold in thresholds:
        y_bin = (smoothed_preds >= threshold).astype(np.int32)
        current_f1 = f1_cust(y_train_oof, y_bin)
        # print(f"{threshold:.4}\t{current_f1:.5}")

        if current_f1 > best_f1:
          best_f1 = current_f1
          # print(f"New best F1:{best_f1:.4}")
          best_window = current_window
          best_threshold = threshold
    # print(f"Current best F1:{best_f1:.4}")
    print(f"Window {current_window}\tCurr best: W:{best_window}\tT:{best_threshold:.4}\tF1:{best_f1:.4}")

oof_pred_adj = smooth_window(oof_pred, window_size=best_window)
oof_pred_adj = (oof_pred_adj >= best_threshold).astype(int)

f1_rez = f1_cust(y_train_oof, oof_pred_adj)
print(f"Window {best_window}, threshold {best_threshold:.4f}:\t{f1_rez:.4f}")

del current_f1, current_window, threshold, f1_rez, window_min, window_max, y_bin, best_f1

## Threshold

In [ ]:
# threshold = 0.5
# y_bin = (oof_pred >= threshold).astype(np.int32)
# best_f1 = f1_score(y_train_oof, y_bin)
# print(f"F1 at default 0.5 threshold: {best_f1:.4f}")

# best_threshold = 0
# thresholds = np.linspace(0.01, 0.99, 100)

# for threshold in thresholds:
#     y_bin = (y_proba >= threshold).astype(np.int32)
#     current_f1 = f1_score(y_train_oof, y_bin)

#     if current_f1 > best_f1:
#         best_f1 = current_f1
#         best_threshold = threshold

# print(f"Optimal threshold:\t{best_threshold:.4f}")
# print(f"Threshold F1:\t{best_f1:.4f}")

In [ ]:
# @title

# X_itest_mini, X_holdout, y_itest_mini, y_holdout = train_test_split(X_test_inner, y_test_inner, test_size=0.2, random_state=42, shuffle=False)

# t_tuner = TunedThresholdClassifierCV(
#     estimator=final_model,
#     scoring=make_scorer(f1_score),
#     cv="prefit",
#     thresholds=100,
#     refit=False,
#     n_jobs=-1
# )

# t_tuner.fit(X_itest_mini, y_itest_mini)

# print(
#     "Threshold:", t_tuner.best_threshold_,
#     "F1:", t_tuner.best_score_
# )

# t_x_pred = t_tuner.predict(X_holdout)

# print(
#     f1_score(y_holdout, t_x_pred),
#     accuracy_score(y_holdout, t_x_pred),
#     precision_score(y_holdout, t_x_pred),
#     recall_score(y_holdout, t_x_pred),
#     sep="\n"
# )

In [ ]:
# @title
# # Thresholds
# X_itest_train, X_itest_test, y_itest_train, y_itest_test = train_test_split(X_test_inner, y_test_inner, test_size=0.2, shuffle=False)

# threshold = 0.5
# y_proba = final_model.predict_proba(X_itest_train)[:,1]
# y_bin = (y_proba >= threshold).astype(np.int32)
# best_f1 = f1_score(y_itest_train, y_bin)
# print(f"F1 at default 0.5 threshold: {best_f1:.4f}")

# best_threshold = 0
# thresholds = np.linspace(0.01, 0.99, 100)

# for threshold in thresholds:
#     y_bin = (y_proba >= threshold).astype(np.int32)
#     current_f1 = f1_score(y_itest_train, y_bin)

#     if current_f1 > best_f1:
#         best_f1 = current_f1
#         best_threshold = threshold

# print(f"Optimal threshold:\t{best_threshold:.4f}")
# print(f"Threshold F1:\t{best_f1:.4f}")

# y_proba_test = final_model.predict_proba(X_itest_test)[:,1]
# final_predictions = (y_proba_test >= best_threshold).astype(np.int32)

# print(
#     "\n--- Metrics on Holdout Set (using optimal threshold) ---",
#     f"F1:\t{f1_score(y_itest_test, final_predictions):.4f}",
#     f"Acc:\t{accuracy_score(y_itest_test, final_predictions):.4f}",
#     f"Pre:\t{precision_score(y_itest_test, final_predictions):.4f}",
#     f"Rec:\t{recall_score(y_itest_test, final_predictions):.4f}",
#     sep="\n"
# )
# # print(f"F1:\t{f1_score(y_itest_test, final_predictions):.4f}")
# # print(f"Acc:\t{accuracy_score(y_itest_test, final_predictions):.4f}")
# # print(f"Pre:\t{precision_score(y_itest_test, final_predictions):.4f}")
# # print(f"Rec:\t{recall_score(y_itest_test, final_predictions):.4f}")

# del y_proba, threshold, thresholds, best_f1
# del X_itest_train, y_itest_train

## Smoothing

To smooth the post-processing results, windows can be tested to determine the optimal majority-vote,

This function will run on CPU.

In [ ]:
# def apply_smoothing(predictions_array, window_size=5):
#     predictions_series = pd.Series(predictions_array)
#     smoothed = predictions_series.rolling(
#         window=window_size,
#         center=True,
#         min_periods=1
#     ).apply(lambda x: np.bincount(x.astype(int)).argmax(), raw=False).astype(int)
#     return smoothed.values


# # Smallest window size
# window_min = 3
# # Largest window size
# window_max = 25

# # Start w no smoothing
# best_window_size = 1
# best_smoothing_f1 = f1_score(y_itest_test, final_predictions)
# # print("No window F1:\t", best_smoothing_f1)
# print(f"No window F1:\t{best_smoothing_f1:.4f}")

# # Test range of odd window sizes
# for window_size in range(window_min, window_max+1, 2):
#     # print("Testing window =", window_size)
#     smoothed_preds = apply_smoothing(final_predictions, window_size=window_size)
#     f1 = f1_score(y_itest_test, smoothed_preds)
#     # print("Window", window_size, "F1:\t", f1)
#     print(f"Window {window_size} F1:\t{f1:.4f}")

#     if f1 > best_smoothing_f1:
#         # print("F1 improved with smoothing!")
#         best_smoothing_f1 = f1
#         best_window_size = window_size

# print(f"Optimal window size: {best_window_size}")
# print(f"Best F1: {best_smoothing_f1:.4f}")

# del window_min, window_max, f1, window_size

In [ ]:
# def apply_smoothing(predictions_array, window_size=5):
#     predictions_series = pd.Series(predictions_array)
#     smoothed = predictions_series.rolling(
#         window=window_size,
#         center=True,
#         min_periods=1
#     ).apply(lambda x: np.bincount(x.astype(int)).argmax(), raw=False).astype(int)
#     return smoothed.values

# # def apply_smoothing(predictions_array, window_size=5):
# #     # 1. Convert NumPy array to CuPy array (if it isn't already on the GPU)
# #     predictions_cp = cp.asarray(predictions_array)

# #     # 2. Convert CuPy array to cuDF Series
# #     predictions_series_gpu = cudf.Series(predictions_cp)

# #     # Define the rolling operation
# #     smoothed_gpu = predictions_series_gpu.rolling(
# #         window=window_size,
# #         center=True,
# #         min_periods=1
# #     )

# #     # 3. Apply the majority vote function using CuPy operations within a custom apply
# #     # We define a custom function using cupy's bincount
# #     def majority_vote_gpu(x):
# #         # Convert to int, count occurrences, and find the index of the max count (argmax)
# #         return cp.bincount(x.astype(cp.int32)).argmax()

# #     # Apply the function
# #     # Note: cuDF's rolling apply works best with simple cupy aggregations.
# #     # For complex custom lambdas like this, performance may vary,
# #     # but it keeps the data on the GPU.
# #     smoothed_gpu = smoothed_gpu.apply(majority_vote_gpu)

# #     # 4. Convert back to CPU NumPy array for final use if needed (optional)
# #     return smoothed_gpu.values.get() # .get() moves data from GPU (CuPy array) back to CPU (NumPy array)
# #     # return smoothed_gpu.values # Keep as CuPy array on GPU

In [ ]:
# # Smallest window size
# window_min = 3
# # Largest window size
# window_max = 25

# # Start w no smoothing
# best_window_size = 1
# best_smoothing_f1 = f1_score(y_itest_test, final_predictions)
# # print("No window F1:\t", best_smoothing_f1)
# print(f"No window F1:\t{best_smoothing_f1:.4f}")

# # Test range of odd window sizes
# for window_size in range(window_min, window_max+1, 2):
#     # print("Testing window =", window_size)
#     smoothed_preds = apply_smoothing(final_predictions, window_size=window_size)
#     f1 = f1_score(y_itest_test, smoothed_preds)
#     # print("Window", window_size, "F1:\t", f1)
#     print(f"Window {window_size} F1:\t{f1:.4f}")

#     if f1 > best_smoothing_f1:
#         # print("F1 improved with smoothing!")
#         best_smoothing_f1 = f1
#         best_window_size = window_size

# print(f"Optimal window size: {best_window_size}")
# print(f"Best F1: {best_smoothing_f1:.4f}")

# del window_min, window_max, f1, window_size

## Applying to test set

In [ ]:
final_model.fit(X_train, y_train)

final_pred_y = final_model.predict_proba(X_test)[:,1]

final_pred_y_base = (final_pred_y >= 0.5).astype(np.int32)

y_test_conv = y_test.copy().to_cupy()
y_test_conv = y_test_conv.get()


print("Base model (0.5 threshold, no smoothing)")

print(
    f"F1:\t{f1_score(y_test_conv, final_pred_y_base):.4f}",
    f"Acc:\t{accuracy_score(y_test_conv, final_pred_y_base):.4f}",
    f"Pre:\t{precision_score(y_test_conv, final_pred_y_base):.4f}",
    f"Rec:\t{recall_score(y_test_conv, final_pred_y_base):.4f}",
    "-----------------------------------",
    sep="\n"
)

# t_x_pred_f = t_tuner.predict(X_test)
print("Windowed with", best_window)

final_pred_y_win = smooth_window(final_pred_y, window_size=best_window)
final_pred_y_win = (final_pred_y_win >= 0.5).astype(np.int32)

print(
    f"F1:\t{f1_score(y_test_conv, final_pred_y_win):.4f}",
    f"Acc:\t{accuracy_score(y_test_conv, final_pred_y_win):.4f}",
    f"Pre:\t{precision_score(y_test_conv, final_pred_y_win):.4f}",
    f"Rec:\t{recall_score(y_test_conv, final_pred_y_win):.4f}",
    "-----------------------------------",
    sep="\n"
)


print(f"Optmized threshold of {best_threshold:.4f}")

# final_pred_y = (final_pred_y >= best_threshold).astype(np.int32)
final_pred_y_opt = smooth_window(final_pred_y, window_size=best_window)
final_pred_y_opt = (final_pred_y_opt >= best_threshold).astype(np.int32)

print(
    f"F1:\t{f1_score(y_test_conv, final_pred_y_opt):.4f}",
    f"Acc:\t{accuracy_score(y_test_conv, final_pred_y_opt):.4f}",
    f"Pre:\t{precision_score(y_test_conv, final_pred_y_opt):.4f}",
    f"Rec:\t{recall_score(y_test_conv, final_pred_y_opt):.4f}",
    "-----------------------------------",
    sep="\n"
)

In [ ]:
play_chime()

## Feature selection

In [ ]:
feature_importances = final_model.feature_importances_
# map scores to feature names
# feature_importances
feature_names = X_train.columns.tolist()

feature_importance_df = pd.DataFrame({
    'feat': feature_names,
    'importance': feature_importances
})

# sort importance
feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

# print(feature_importance_df)
feature_importance_df

# most important features
# print(feature_importance_df.head(25))

In [ ]:
threshold_importance = 0.95
# calculate most important 90 percent of the importance
feature_importance_df['cumulative_imp'] = feature_importance_df['importance'].cumsum()
features_percent = feature_importance_df[feature_importance_df['cumulative_imp'] <= threshold_importance].shape[0] + 1
features_percent

In [ ]:
# Most important features:
print(round(threshold_importance*100), "% (most important features):", features_percent)
feature_importance_df.head(features_percent)

In [ ]:
# feature_importance_df.tail(1)
# Least important features:
print("Remaining", round((1-threshold_importance)*100), "% (least important features):", len(feature_names)-features_percent)
feature_importance_df.tail(len(feature_names)-features_percent)

In [ ]:
# Features with 0 importance:
print("Features with 0 importance:", len(feature_importance_df[feature_importance_df['importance']==0]))

feature_importance_df[feature_importance_df['importance']==0]

Feature importance by type

In [ ]:
mapping_dict = {
    'soil': '_deep|_shallow',
    'runoff':'ro',
    'rain':'rain',
    'calibration':'_cal'
}

for col_name, pattern in mapping_dict.items():
    feature_importance_df[col_name] = feature_importance_df['feat'].str.contains(pattern, case=False, regex=True)

feature_importance_df

In [ ]:
feature_importance_df['most'] = (feature_importance_df['cumulative_imp'] <= threshold_importance)
feature_importance_df['zero'] = (feature_importance_df['importance'] == 0)

cat_cols = list(mapping_dict.keys())

table_feature_cat_importance = pd.DataFrame({
    'Total features': feature_importance_df[cat_cols].sum(),
    'Above threshold': feature_importance_df[feature_importance_df['most']][cat_cols].sum(),
    'Below threshold': feature_importance_df[~feature_importance_df['most']][cat_cols].sum(),
    'Zero importance': feature_importance_df[feature_importance_df['zero']][cat_cols].sum()
}).fillna(0).astype(int)

del cat_cols, mapping_dict

table_feature_cat_importance.index.name = 'Category'

table_feature_cat_importance

## Classification Threshold

By default, a threshold of 0.5 will be selected for categorizing a point as `True` or `False`. However, in this context a model more sensitive to `True` may make final results more accurate.

This measure can be found by finding the threshold that maximizes the F1 score.

In [ ]:
# classifier_tuned = TunedThresholdClassifierCV(best_model, scoring="balanced_accuracy").fit(X_train_two, y_train_two)
# print(f"Cut-off point found at {classifier_tuned.best_threshold_:.3f}")
t_tuner = TunedThresholdClassifierCV(
    estimator=best_model,
    scoring=make_scorer(f1_score),
    cv="prefit",
    thresholds=100,
    refit=False
)

t_tuner.fit(X_test_inner, y_test_inner)

print(
    "Threshold:", t_tuner.best_threshold_,
    "F1:", t_tuner.best_score_
)

In [ ]:
t_x_pred = t_tuner.predict(X_test_inner)

print(
    f1_score(y_test_inner, t_x_pred),
    accuracy_score(y_test_inner, t_x_pred),
    precision_score(y_test_inner, t_x_pred),
    recall_score(y_test_inner, t_x_pred),
    sep="\n"
)

In [ ]:
_ = playsound(get_path('completed.mp3', 'code'), block=False)
# X_train_two, X_val, y_train_two, y_val = train_test_split(X_train, y_train, test_size = 0.2, shuffle=False)

In [ ]:
all_f1 = []
all_accuracy = []
all_precision = []
all_recall = []
optimal_thresholds = []
performance_df = {}
# performance_df = DataFrame()

for fold, (index_train, index_val) in enumerate(tscv.split(X_train)):
    print("Split", fold)
    X_sub_train, X_sub_val = X_train.iloc[index_train].copy(), X_train.iloc[index_val].copy()
    y_sub_train, y_sub_val = y_train.iloc[index_train].copy(), y_train.iloc[index_val].copy()

    # y_sub_train = y_sub_train.drop(i_to_drop, errors='ignore')

    # if len(y_sub_train.unique()) != 2:
    #     print("Skipping fold", fold)
    #     continue

    # model = xgb.XGBClassifier(
    #     objective='binary:logistic',
    #     eval_metric='logloss',
    #     tree_method = "hist",
    #     random_state = 42,
    #     scale_pos_weight = (y_sub_train.value_counts()[False] / y_sub_train.value_counts()[True]).item()
    # )

    print("Fitting model...")
    best_model.fit(
        X_sub_train, y_sub_train,
        # Evaluation set
        eval_set = [(X_sub_val, y_sub_val)],
        # Weight of False vs True
        # early_stopping_rounds = 50,
        # Silence messages
        verbose=False
    )

    print("Predicting probabilities...")
    # Get probabilities for the positive class (class 1)
    y_pred_proba = best_model.predict_proba(X_sub_val)[:, 1]

    # 1. Calculate precision, recall, and thresholds for the current fold
    precision, recall, thresholds = precision_recall_curve(y_sub_val, y_pred_proba)

    # 2. Find the optimal threshold based on F1-score (or whichever metric you prefer)
    fscores = (2 * precision * recall) / (precision + recall)
    fscores[np.isnan(fscores)] = 0
    optimal_idx = np.argmax(fscores)

    # Need to handle the fact that thresholds array is one element shorter than P/R/F1 arrays
    # Best practice is often to use the threshold just after the optimal index is found in P/R/F1 arrays
    best_threshold = thresholds[optimal_idx]
    optimal_thresholds.append(best_threshold)

    print(f"Fold {fold} Optimal Threshold (Max F1): {best_threshold:.4f}")

    # 3. Apply the *optimal* threshold to the validation predictions for THIS fold
    y_pred_optimal = (y_pred_proba >= best_threshold).astype(int)

    # 4. Calculate metrics using the *optimally thresholded* predictions
    print("Getting metrics using optimal threshold...")
    fold_f1 = f1_score(y_sub_val, y_pred_optimal)
    fold_accuracy = accuracy_score(y_sub_val, y_pred_optimal)
    fold_precision = precision_score(y_sub_val, y_pred_optimal)
    fold_recall = recall_score(y_sub_val, y_pred_optimal)

    all_f1.append(fold_f1)
    all_accuracy.append(fold_accuracy)
    all_precision.append(fold_precision)
    all_recall.append(fold_recall)

    # You might want to use a more structured DataFrame for performance_df construction
    performance_df[fold] = {
        'Threshold': best_threshold,
        'F1': fold_f1,
        'Accuracy': fold_accuracy,
        'Precision': fold_precision,
        'Recall': fold_recall
    }

    print(f"{fold}\tOptimal F1: {fold_f1:.4f}\tAcc: {fold_accuracy:.4f}\tPrec: {fold_precision:.4f}\tRec: {fold_recall:.4f}")
    # print("Predicting...")
    # y_pred = best_model.predict(X_sub_val)
    # y_pred_proba = best_model.predict_proba(X_sub_val)[:, 1]

    # # 2. Calculate precision, recall, and thresholds for the current fold
    # precision, recall, thresholds = precision_recall_curve(y_sub_val, y_pred_proba)

    # # 3. Find the optimal threshold based on F1-score (a common choice for balance)
    # # Calculate F1-score for every possible threshold
    # fscores = (2 * precision * recall) / (precision + recall)

    # # Handle potential division by zero warnings if no positive predictions were made
    # fscores[np.isnan(fscores)] = 0

    # # Locate the index of the highest F1-score
    # optimal_idx = np.argmax(fscores)
    # best_threshold = thresholds[optimal_idx] # Note: thresholds array is one element shorter than P/R arrays

    # optimal_thresholds.append(best_threshold)

    # print(f"Fold {fold} Optimal Threshold (Max F1): {best_threshold:.4f}")


    # print("Getting metrics...")
    # fold_f1 = f1_score(y_sub_val, y_pred)
    # fold_accuracy = accuracy_score(y_sub_val, y_pred)
    # fold_precision = precision_score(y_sub_val, y_pred)
    # fold_recall = recall_score(y_sub_val, y_pred)

    # all_f1.append(fold_f1)
    # all_accuracy.append(fold_accuracy)
    # all_precision.append(fold_precision)
    # all_recall.append(fold_recall)
    # performance_df[fold] = {best_threshold, fold_f1, fold_accuracy, fold_precision, fold_recall}

    # print(f"{fold}\tF1: {fold_f1:.4f}\tAcc{fold_accuracy:.4f}\tPrec{fold_precision:.4f}\tRec: {fold_recall:.4f}")

print(performance_df)

In [ ]:
_ = playsound(get_path('completed.mp3', 'code'), block=False)

In [ ]:
# import pprint

# pprint.pprint(performance_df)